In [1]:
import os
import cv2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from efficientnet_pytorch import EfficientNet
from albumentations import Compose, Normalize
from albumentations.pytorch import ToTensorV2

In [ ]:
class Config:
    # Data directories
    TRAIN_IMG_DIR = "D:/HUS_third_year/ki_2/TGMT/project/project/archive/ISIC_2019_Training_Input/processed_train_isic2019"
    TEST_IMG_DIR = "D:/HUS_third_year/ki_2/TGMT/project/project/archive/test/processed_test_isic2019_2"
    TRAIN_GT_CSV = "D:/HUS_third_year/ki_2/TGMT/project/project/archive/ISIC_2019_Training_GroundTruth.csv"
    TEST_GT_CSV = "D:/HUS_third_year/ki_2/TGMT/project/project/ISIC_2019_Test_GroundTruth.csv"
    TRAIN_META_CSV = "D:/HUS_third_year/ki_2/TGMT/project/project/archive/ISIC_2019_Training_Metadata.csv"
    TEST_META_CSV = "D:/HUS_third_year/ki_2/TGMT/project/project/ISIC_2019_Test_Metadata.csv"
    OUTPUT_DIR = 'D:/HUS_third_year/ki_2/TGMT/project/project/outputs/task2'
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # Training parameters
    IMG_SIZE = 600
    BATCH_SIZE = 16
    EPOCHS = 30  # Reduced for metadata training
    LR = 1e-3   # Lower LR for metadata training
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

    # Classes (8 diagnoses + UNK)
    CLASSES = ['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC', 'UNK']
    N_CLASSES = len(CLASSES)

    # Metadata parameters
    ANATOMICAL_SITES = ['head/neck', 'upper extremity', 'lower extremity', 'torso', 'palms/soles', 'oral/genital', 'lateral torso', 'anterior torso']
    N_SITES = len(ANATOMICAL_SITES)
    N_SEX = 2  # Male, Female
    META_FEATURES = N_SITES + N_SEX + 1  # Sites + Sex + Age

In [3]:
class Task2Dataset(Dataset):
    def __init__(self, df_gt, df_meta, img_dir, transforms=None, meta_dropout_prob=0.1):
        """Dataset loads preprocessed images and metadata, applies transforms and metadata encoding"""
        self.df = df_gt.merge(df_meta, on='image_id', how='left').reset_index(drop=True)
        self.img_dir = img_dir
        self.transforms = transforms
        self.meta_dropout_prob = meta_dropout_prob
        self.site_to_idx = {site: i for i, site in enumerate(Config.ANATOMICAL_SITES)}

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.loc[idx]
        img_path = os.path.join(self.img_dir, row['image_id'] + '.jpg')
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Metadata encoding
        site = row.get('anatomical_site_general', np.nan)
        sex = row.get('sex', np.nan)
        age = row.get('age_approx', np.nan)

        # One-hot encoding for anatomical site
        site_vec = np.zeros(Config.N_SITES, dtype=np.float32)
        if isinstance(site, str) and np.random.rand() > self.meta_dropout_prob:
            site_vec[self.site_to_idx.get(site, 0)] = 1.0

        # One-hot encoding for sex
        sex_vec = np.zeros(Config.N_SEX, dtype=np.float32)
        if isinstance(sex, str) and np.random.rand() > self.meta_dropout_prob:
            sex_vec[0 if sex == 'male' else 1] = 1.0

        # Numerical encoding for age
        age_val = -5.0 if np.isnan(age) or np.random.rand() < self.meta_dropout_prob else float(age)

        meta = np.concatenate([site_vec, sex_vec, [age_val]], axis=0)

        if self.transforms:
            img = self.transforms(image=img)['image']

        return img, torch.tensor(meta, dtype=torch.float32), row['label']

In [4]:
class Task2Net(nn.Module):
    def __init__(self, backbone='efficientnet-b3', pretrained_model_path=None):
        super().__init__()
        # Load pretrained CNN and freeze weights
        self.cnn = EfficientNet.from_name(backbone)
        if pretrained_model_path:
            state_dict = torch.load(pretrained_model_path)
            self.cnn.load_state_dict(state_dict)
        for param in self.cnn.parameters():
            param.requires_grad = False

        in_f = self.cnn._fc.in_features
        self.cnn._fc = nn.Identity()  # Remove final FC layer

        # Metadata network
        self.meta_net = nn.Sequential(
            nn.Linear(Config.META_FEATURES, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(256, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.4)
        )

        # Combined network
        self.combined = nn.Sequential(
            nn.Linear(in_f + 256, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(1024, Config.N_CLASSES)
        )

    def forward(self, img, meta):
        img_features = self.cnn(img)
        meta_features = self.meta_net(meta)
        combined = torch.cat([img_features, meta_features], dim=1)
        return self.combined(combined)


In [5]:
def train_one_epoch(model, loader, criterion, optimizer):
    model.train()
    total_loss = 0
    for imgs, metas, labels in loader:
        imgs, metas, labels = imgs.to(Config.DEVICE), metas.to(Config.DEVICE), labels.to(Config.DEVICE)
        optimizer.zero_grad()
        outputs = model(imgs, metas)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * imgs.size(0)
    return total_loss / len(loader.dataset)

In [6]:
def evaluate(model, loader, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    with torch.no_grad():
        for imgs, metas, labels in loader:
            imgs, metas, labels = imgs.to(Config.DEVICE), metas.to(Config.DEVICE), labels.to(Config.DEVICE)
            outputs = model(imgs, metas)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * imgs.size(0)
            correct += (outputs.argmax(1) == labels).sum().item()
    return total_loss / len(loader.dataset), correct / len(loader.dataset)

In [ ]:
# Load ground-truth and metadata CSVs
train_gt = pd.read_csv(Config.TRAIN_GT_CSV)
train_gt.rename(columns={'image': 'image_id'}, inplace=True)
train_gt['label'] = train_gt[Config.CLASSES].values.argmax(axis=1)

In [ ]:
test_gt = pd.read_csv(Config.TEST_GT_CSV)
test_gt.rename(columns={'image': 'image_id'}, inplace=True)
test_gt['label'] = test_gt[Config.CLASSES].values.argmax(axis=1)

In [ ]:
train_meta = pd.read_csv(Config.TRAIN_META_CSV)
test_meta = pd.read_csv(Config.TEST_META_CSV)

In [ ]:
# Transforms
transforms = Compose([Normalize(), ToTensorV2()])

In [ ]:
# DataLoaders
train_loader = DataLoader(
    Task2Dataset(train_gt, train_meta, Config.TRAIN_IMG_DIR, transforms=transforms),
    batch_size=Config.BATCH_SIZE, shuffle=True, num_workers=4)

In [ ]:
test_loader = DataLoader(
    Task2Dataset(test_gt, test_meta, Config.TEST_IMG_DIR, transforms=transforms, meta_dropout_prob=0.0),
    batch_size=Config.BATCH_SIZE, shuffle=False, num_workers=4)

In [ ]:
# Model, load pretrained Task 1 weights
model = Task2Net(pretrained_model_path=os.path.join('D:/HUS_third_year/ki_2/TGMT/project/project/outputs/task1', 'task1_final.pth')).to(Config.DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam([p for p in model.parameters() if p.requires_grad], lr=Config.LR)

In [ ]:
# Training loop
for epoch in range(Config.EPOCHS):
    train_loss = train_one_epoch(model, train_loader, criterion, optimizer)
    print(f"Epoch {epoch}: Train Loss={train_loss:.4f}")

In [ ]:
# Save model
torch.save(model.state_dict(), os.path.join(Config.OUTPUT_DIR, 'task2_final.pth'))

In [ ]:
# Evaluate on test set
model.load_state_dict(torch.load(os.path.join(Config.OUTPUT_DIR, 'task2_final.pth')))
test_loss, test_acc = evaluate(model, test_loader, criterion)
print(f"Test Loss={test_loss:.4f}, Test Acc={test_acc:.4f}")